In [1]:
# Import stuff
from os.path import join
from pandas import DataFrame

from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.utility import IdentityInterface, Function
from nipype.interfaces.io import DataSink, FreeSurferSource, SelectFiles
from nipype.algorithms.misc import Gunzip

from nipype.interfaces.freesurfer.preprocess import MRIConvert
from nipype.interfaces.freesurfer.model import Binarize
from nipype.interfaces.freesurfer import FSCommand, MRIConvert
from nipype.interfaces.fsl.utils import Reorient2Std, MotionOutliers
from nipype.interfaces.fsl.preprocess import MCFLIRT, SliceTimer, FLIRT
from nipype.interfaces.fsl.maths import ApplyMask
from nipype.interfaces.fsl.model import GLM
from nipype.algorithms.rapidart import ArtifactDetect

# FSL set up- change default file output type
from nipype.interfaces.fsl import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

# Set study variables
analysis_home = '/home/camachocm2/Analysis/ChEC/fmri_proc'
raw_dir = analysis_home + '/raw'
preproc_dir = analysis_home + '/preproc'
firstlevel_dir = analysis_home + '/subjectlevel'
secondlevel_dir = analysis_home + '/grouplevel'
workflow_dir = analysis_home + '/workflows'

#subject_info = DataFrame.read_csv(analysis_home + '/../misc/subjs.csv')
#subjects_list = subject_info['SubjID'].tolist()
subjects_list = ['pilot002']

# FreeSurfer set up - change SUBJECTS_DIR 
fs_dir = preproc_dir + '/freesurfer_dir'
FSCommand.set_default_subjects_dir(fs_dir)

# data collection specs
TR = 0.8 #in seconds
num_slices = 40
slice_direction = False #True = z direction, top to bottom
interleaved = True

In [2]:
## Data handling Nodes

# Select subjects list
infosource = Node(IdentityInterface(fields=['subjid']),
                  name='infosource')
infosource.iterables = [('subjid', subjects_list)]

# FreeSurferSource - Data grabber specific for FreeSurfer data
fssource = Node(FreeSurferSource(subjects_dir=fs_dir),
                run_without_submitting=True,
                name='fssource')

# Sink data
substitutions = [('_subjid_', '')] #output file name substitutions
datasink = Node(DataSink(base_directory = preproc_dir,
                        container = preproc_dir,
                        substitutions = substitutions), 
                name='datasink')

## fMRI data Reconstruction Workflow

In [3]:
# Pull files
rawfile_template = {'folder': raw_dir + '/{subjid}/{func}'}
recon_selectfiles = Node(SelectFiles(rawfile_template), name='recon_selectfiles')
recon_selectfiles.iterables = [('func',['chec_movie_PA','corr_chec_1','corr_chec_2'])]

In [8]:
## reconstruction nodes

def sort_dicoms(dicom_directory):    
    from pydicom import dcmread
    from pandas import DataFrame
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from glob import glob
    from os.path import isdir
    from os import makedirs
    import shutil
    
    dicoms = glob(dicom_directory + '/MR.*')
    dicoms = sorted(dicoms)
     
    for dicom in dicoms:
        info = dcmread(dicom)
        te = info[0x18,0x81].value
        
        if not isdir(dicom_directory + '/TE_%d' % te):
                makedirs(dicom_directory + '/TE_%d' % te)

        shutil.move(dicom, dicom_directory + '/TE_%d' % te)
    
    dicom_names = []
    TE_folders = glob(dicom_directory + '/TE_*')
    TE_folders = sorted(TE_folders)
    for folder in TE_folders:
        temp = glob(folder + '/MR*')
        temp = sorted(temp)
        dicom_names.append(temp[0])
        
    return(dicom_names)


In [9]:
# Pull Dicom information
dicom_info = Node(Function(input_names=['dicom_directory'],
                           output_names=['dicom_names'],  
                           function=sort_dicoms), 
                       name='main_dicom_info')

# convert to niftis
convert_to_niftis = MapNode(MRIConvert(out_file='func.nii.gz', 
                                       out_type='niigz'), 
                            name='convert_to_niftis', 
                            iterfield=['in_file'])

In [10]:
recon_flow = Workflow(name='recon_flow')
recon_flow.connect([(infosource, recon_selectfiles,[('subjid','subjid')]), 
                    (recon_selectfiles, dicom_info,[('folder','dicom_directory')]), 
                    (dicom_info, convert_to_niftis,[('dicom_names','in_file')]),
                    
                    (convert_to_niftis, datasink, [('out_file','separated_niftis')])
                   ])

recon_flow.base_dir = workflow_dir
recon_flow.write_graph(graph2use='flat')
recon_flow.run('MultiProc', plugin_args={'n_procs': 4, 'memory_gb':10})

180619-14:09:22,124 workflow INFO:
	 Generated workflow graph: /home/camachocm2/Analysis/ChEC/fmri_proc/workflows/recon_flow/graph.png (graph2use=flat, simple_form=True).
180619-14:09:22,156 workflow INFO:
	 Workflow recon_flow settings: ['check', 'execution', 'logging', 'monitoring']
180619-14:09:22,201 workflow INFO:
	 Running in parallel.
180619-14:09:22,205 workflow INFO:
	 [MultiProc] Running 0 tasks, and 3 jobs ready. Free memory (GB): 10.00/10.00, Free processors: 4/4.
180619-14:09:22,291 workflow INFO:
	 [Node] Setting-up "recon_flow.recon_selectfiles" in "/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/recon_flow/_subjid_pilot002/_func_chec_movie_PA/recon_selectfiles".180619-14:09:22,293 workflow INFO:
	 [Node] Setting-up "recon_flow.recon_selectfiles" in "/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/recon_flow/_subjid_pilot002/_func_corr_chec_1/recon_selectfiles".180619-14:09:22,292 workflow INFO:
	 [Node] Setting-up "recon_flow.recon_selectfiles" in "/home/camachocm

## fMRI Data Preprocessing Workflows

In [ ]:
## File handling nodes

preproc_template={'func_te13': preproc_dir + '/separated_niftis/{subjid}/_func_chec_movie_PA/_convert_to_niftis0/func.nii.gz', 
                  'func_te38': preproc_dir + '/separated_niftis/{subjid}/_func_chec_movie_PA/_convert_to_niftis1/func.nii.gz', 
                  'func_te64': preproc_dir + '/separated_niftis/{subjid}/_func_chec_movie_PA/_convert_to_niftis2/func.nii.gz', 
                  'func_AP': preproc_dir + '/separated_niftis/{subjid}/_func_corr_chec_1/_convert_to_niftis0/func.nii.gz', 
                  'func_PA':preproc_dir + '/separated_niftis/{subjid}/_func_corr_chec_2/_convert_to_niftis0/func.nii.gz'}
preproc_selectfiles = Node(SelectFiles(preproc_template), name='preproc_selectfiles')

In [ ]:
## fMRI Data processing nodes

# reorient images to MNI space standard
reorientFunc = Node(Reorient2Std(terminal_output='file'),
                   name='reorientFunc')

# perform slice time correction
slicetime = Node(SliceTimer(index_dir=slice_direction,
                           interleaved=interleaved,
                           time_repetition=TR),
                name='slicetime')

# realignment using mcflirt
realignmc = Node(MCFLIRT(save_plots=True),
                name='realignmc')

# Coregistration using flirt
coregflt = Node(FLIRT(),
               name='coregflt')
coregflt2 = Node(FLIRT(apply_xfm=True, 
                       out_file='preproc_func.nii.gz'),
                name='coregflt2')

#unzip file before feeding into ART
gunzip_mask = Node(Gunzip(), name='gunzip_mask')
gunzip_func = Node(Gunzip(), name='gunzip_func')

# Artifact detection for scrubbing/motion assessment
art = Node(ArtifactDetect(mask_type='file',
                          parameter_source='FSL',
                          bound_by_brainmask=True,
                          norm_threshold=1,
                          zintensity_threshold=3,
                          use_differences=[True, False]),
           name='art')

In [ ]:
## Anatomical processing
# Convert skullstripped brain to nii, resample to 2mm^3
resample = Node(MRIConvert(out_type='nii',
                          vox_size=(2,2,2)),
               name='resample')

# Reorient anat to MNI space
reorientAnat = Node(Reorient2Std(terminal_output='file'),
                   name='reorientAnat')

# Create binary mask of resampled, skullstripped anat, dilate, and erode to fill gaps
binarize = Node(Binarize(dilate=1,
                        erode=1,
                        invert=False,
                        min=1,
                        max=255),
               name='binarize')

In [ ]:
# Data QC nodes
def create_coreg_plot(epi,anat):
    import os
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nilearn import plotting
    
    coreg_filename='coregistration.png'
    display = plotting.plot_anat(epi, display_mode='ortho',
                                 draw_cross=False,
                                 title = 'coregistration to anatomy')
    display.add_edges(anat)
    display.savefig(coreg_filename) 
    display.close()
    coreg_file = os.path.abspath(coreg_filename)
    
    return(coreg_file)

def check_mask_coverage(epi,brainmask):
    import os
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nilearn import plotting
    
    maskcheck_filename='maskcheck.png'
    display = plotting.plot_anat(epi, display_mode='ortho',
                                 draw_cross=False,
                                 title = 'brainmask coverage')
    display.add_contours(brainmask,levels=[.5], colors='r')
    display.savefig(maskcheck_filename)
    display.close()
    maskcheck_file = os.path.abspath(maskcheck_filename)

    return(maskcheck_file)

make_coreg_img = Node(name='make_coreg_img',
                      interface=Function(input_names=['epi','anat'],
                                         output_names=['coreg_file'],
                                         function=create_coreg_plot))

make_checkmask_img = Node(name='make_checkmask_img',
                      interface=Function(input_names=['epi','brainmask'],
                                         output_names=['maskcheck_file'],
                                         function=check_mask_coverage))

In [ ]:
preprocflow = Workflow(name='preprocflow')

preprocflow.connect([(infosource, selectfunc, [('subjid','subjid')]),
                     (selectfunc,reorientFunc, [('func','in_file')]),
                     (infosource, fssource, [('subjid','subject_id')]),
                     (fssource, resample, [('brainmask','in_file')]),
                     (resample, reorientAnat, [('out_file','in_file')]),
                     (reorientAnat, binarize, [('out_file','in_file')]),
                     (reorientFunc, slicetime, [('out_file','in_file')]),
                     (slicetime, realignmc, [('slice_time_corrected_file','in_file')]),
                     (reorientAnat, coregflt, [('out_file','reference')]),
                     (realignmc, coregflt, [('out_file','in_file')]),
                     (realignmc, coregflt2, [('out_file','in_file')]),
                     (coregflt, coregflt2, [('out_matrix_file','in_matrix_file')]),
                     (reorientAnat, coregflt2, [('out_file','reference')]),
                     (binarize, gunzip_mask,[('binary_file','in_file')]), 
                     (gunzip_mask, art, [('out_file','mask_file')]),
                     (coregflt2, gunzip_func, [('out_file','in_file')]),
                     (gunzip_func, art, [('out_file','realigned_files')]),
                     (realignmc, art, [('par_file','realignment_parameters')]),
                     (coregflt, make_coreg_img, [('out_file','epi')]),
                     (reorientAnat, make_coreg_img, [('out_file','anat')]),
                     (coregflt, make_checkmask_img, [('out_file','epi')]),
                     (binarize, make_checkmask_img, [('binary_file','brainmask')]),
                     
                     (reorientAnat, datasink, [('out_file','reoriented_anat')]),
                     (binarize, datasink, [('binary_file','binarized_anat')]),
                     (make_checkmask_img, datasink, [('maskcheck_file','checkmask_image')]),
                     (make_coreg_img, datasink, [('coreg_file','coreg_image')]),
                     (coregflt2, datasink, [('out_file','coreg_func')]),
                     (art, datasink, [('plot_files','art_plot')]), 
                     (art, datasink, [('outlier_files','art_outliers')]),
                     (realignmc, datasink, [('par_file','mcflirt_displacement')])
                    ])
preprocflow.base_dir = workflow_dir
preprocflow.write_graph(graph2use='flat')
preprocflow.run('MultiProc', plugin_args={'n_procs': 4, 'memory_gb':10})

In [ ]:


def make_nifti(dicom, output_dir, volume_name):
    from pandas import DataFrame
    from os.path import basename
    import shutil
    from nipype.interfaces.freesurfer.preprocess import MRIConvert
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    
    temp_path = output_dir + '/' + basename(dicom)
    
    shutil.move(dicom,temp_path)
    mrc = MRIConvert()
    mrc.inputs.in_file = temp_path
    mrc.inputs.out_file = volume_name
    mrc.run()
    shutil.move(temp_path, dicom)
    return()

In [ ]:
from glob import glob
from pandas import DataFrame

dicoms_folder = '/home/camachocm2/Analysis/ChEC/fmri_proc/raw/pilot002/chec_movie_PA'
output_dir = '/home/camachocm2/Analysis/ChEC/fmri_proc/raw/pilot002/chec_sort'


for volnum in range(0, len(files)):
    volume_name = output_dir + '/vol' + str(volnum).zfill(5) + '.nii.gz'
    
    dicom_info = extract_dicom_info(volnum, files[volnum], dicoms_info, volume_name)
    #make_nifti(files[volnum], output_dir, volume_name)
    
dicom_info.to_csv('/home/camachocm2/Analysis/ChEC/fmri_proc/raw/pilot002/dicom_info.csv')

In [ ]:
from pandas import read_csv
dicom_info = read_csv('/home/camachocm2/Analysis/ChEC/fmri_proc/raw/pilot002/dicom_info.csv')
output_dir = '/home/camachocm2/Analysis/ChEC/fmri_proc/raw/pilot002/chec_sort'

for index, row in dicom_info.iterrows():
    dicom = row['DicomName']
    volume = row['VolumeName']
    make_nifti(files[volnum], output_dir, volume_name)
    

In [ ]:
from nipype.interfaces.fsl.utils import Merge

short_te = dicom_info[dicom_info['TE']==13.2]
mid_te = dicom_info[dicom_info['TE']==38.76]
long_te = dicom_info[dicom_info['TE']==64.32]

mid_te_files = mid_te['VolumeName'].tolist()
me = Merge()
me.inputs.in_files = mid_te_files
me.inputs.dimension = 't'
me.inputs.merged_file = '/home/camachocm2/Analysis/ChEC/fmri_proc/raw/pilot002/chec_movie_midTE.nii.gz'
me.run()